In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

# Set the working directory to the main directory
os.chdir(os.path.dirname(os.path.abspath('')))

# Verify the current working directory
print(os.getcwd())

/home/stef/projects/ai-amg


In [3]:
from music_gen.chord_generator import ChordProgressionGenerator
from music_gen.ableton_controllers import AbletonOSCController, AbletonMetaController
# from chord_generator import ChordProgressionGenerator
from music_gen.helpers import chord_to_midi, transpose_midi_chords

In [25]:
# chord_prog = ChordProgressionGenerator()
controller = AbletonOSCController()

controller.song.set_tempo(200)

# controller.device.set_parameter(0, 0, 0, 0) # controlling frequency of filter

# prog = chord_prog.generate_progression(0.5, 0.5)
# midi_chords, midi_bass = chord_to_midi(prog)
# controller.clip.add_notes(0, 0, midi_chords)
# # controller.song.set_tempo(100) 
# arousal = -0.8 # between -1 and 1
# # Map arousal to arp rate (1-127 range)
# arp_rate = ((arousal + 1) / 2) * (127 - 1) + 1  # Rescale to 1-12
# # track number 2, first device (that is the instrument)
# controller.device.set_parameter(0, 0, 0, 1) # controlling frequency of filter

In [4]:
import numpy as np
from scipy.optimize import linear_sum_assignment

def voice_leading(chord_a, chord_b):
    # Sort the chords
    chord_a_sorted = sorted(chord_a)
    chord_b_sorted = sorted(chord_b)
    
    # Create a cost matrix for pairings
    n = len(chord_a_sorted)
    m = len(chord_b_sorted)
    cost_matrix = np.zeros((n, m))
    
    for i, note_a in enumerate(chord_a_sorted):
        for j, note_b in enumerate(chord_b_sorted):
            # Cost is the minimum distance between two notes, considering wrap-around
            direct_move = abs(note_b - note_a)
            wrap_around = 12 - direct_move
            cost_matrix[i, j] = min(direct_move, wrap_around)
    
    # Solve the assignment problem
    row_indices, col_indices = linear_sum_assignment(cost_matrix)
    
    # Map notes from chord_a to chord_b
    new_chord = [chord_b_sorted[j] for i, j in zip(row_indices, col_indices)]
    
    return new_chord

# Example usage
chord_a = [60, 64, 67, 69]  # C major triad
chord_b = [62, 65, 69, 71]  # D minor triad

result = voice_leading(chord_a, chord_b)
print("Resulting chord with smooth voice leading:", result)

Resulting chord with smooth voice leading: [71, 62, 65, 69]


In [ ]:
import numpy as np

def generate_mode_chords(valence):
    """
    Generate chord sequences based on valence and arousal inputs.

    Parameters:
    - valence (float): Emotional valence, a value between 0 and 1.
    - arousal (float): Emotional arousal, a value between 0 and 1.
    - delay (int): Duration (in seconds) to run the generation.

    Returns:
    - List of generated chord sequences for each bar.
    """
    # Define chord sets and modes
    chordlist = np.array([
        [60, 64, 55, 59],
        [62, 65, 57, 60],
        [64, 55, 59, 62],
        [60, 65, 57, 64],
        [55, 59, 62, 65],
        [57, 60, 64, 55],
        [59, 62, 65, 57]
    ])

    modes = ['lydian', 'ionian', 'mixolydian', 'dorian', 'aeolian', 'phrygian', 'locrian']
    mode_map = [
        [3, 6, 0, 3],    # Lydian
        [0, 3, 4, 0],    # Ionian
        [4, 0, 1, 4],    # Mixolydian
        [1, 4, 5, 1],    # Dorian
        [5, 1, 2, 5],    # Aeolian
        [2, 5, 6, 2],    # Phrygian
        [6, 2, 3, 6]     # Locrian
    ]

    # Build mode set (adjusting pitches by lowering 3 semitones)
    modeset = np.zeros((4, 4, 7))
    for i, mode_indices in enumerate(mode_map):
        for j in range(4):
            modeset[j, :, i] = chordlist[mode_indices[j], :]
    modeset -= 3

    # Determine mode based on valence
    mode_index = 7 - round(valence * 6) - 1
    mode_name = modes[int(mode_index)]
    print(f"Selected Mode: {mode_name}")

    # Generate chord sequence for the duration
    chord_sequence = []
    for bar in range(4):
        chords = modeset[bar, :, mode_index]
        chord_sequence.append(chords)
        print(f"Bar {bar + 1}: Chords {chords}")

    return chord_sequence


# Example usage
valence = 0.2 # Example valence
generated_chords = generate_mode_chords(valence)

Selected Mode: phrygian
Bar 1: Chords [61. 52. 56. 59.]
Bar 2: Chords [54. 57. 61. 52.]
Bar 3: Chords [56. 59. 62. 54.]
Bar 4: Chords [61. 52. 56. 59.]
